In [133]:
import numpy as np
from numpy.random import randn
from itertools import count

np.random.seed(1)

In [154]:
expert_size = 3 # number of experts
emb_size = 5 # embedding size
emb_prime_size = 4 # internal expert size
word_size = 7 # think token count, from unrolled batches possibly

# token count * embedding size
words = randn(word_size, emb_size)

In [155]:
# expert (3) * emb_size (5) * emb_prime_size (4)
experts_w1 = randn(expert_size, emb_size, emb_prime_size)

# expert (3) * emb_prime_size (4) * emb_size (5)
experts_w2 = randn(expert_size, emb_prime_size, emb_size)

In [156]:
# experts (3) * tokens
routed = np.array([
    [1, 1, 0, 0, 0, 0, 0], # first two tokens go to expert 1
    [0, 0, 1, 1, 0, 0, 0], # second two tokens go to expert 2
    [0, 0, 0, 0, 1, 1, 0]  # the very last token doesn't go to any expert(!)
])

In [160]:
b = 0.1

# words(7) * emb_size(4)
total_output = b * words # … so that if a word doesn't go through any expert it will still have some value

# Repeat for every expert, looking at routed do determine which words go to said expert
for n, expert_w1, expert_w2, mask in zip(count(), experts_w1, experts_w2, routed):
    print(f"Expert {n}:")
    
    # select all words where the mask for this expert is > 0
    expert_input = words[mask.nonzero()]
    
    expert_temp = np.matmul(expert_input, expert_w1)
    expert_output = np.matmul(expert_temp, expert_w2)

    # I did assignment here, but could also be addition
    total_output[mask.nonzero()] = expert_output

    print(f"  {expert_input.shape} * {expert_w1.shape} * {expert_w2.shape} = {expert_output.shape}\n")

print(f"Total output: {total_output.shape}")

Expert 0:
  (2, 5) * (5, 4) * (4, 5) = (2, 5)

Expert 1:
  (2, 5) * (5, 4) * (4, 5) = (2, 5)

Expert 2:
  (2, 5) * (5, 4) * (4, 5) = (2, 5)

Total output: (7, 5)
